<a href="https://colab.research.google.com/github/rai8896/Deep_Learning_Lab/blob/main/deep_learning_exp_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber torch numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 940.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 55.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.2 MB/s eta 0:00:00:00:01


In [ ]:
import pdfplumber
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


In [ ]:
pdf_path = "/kaggle/input/poems100/poems-100.pdf"   # Colab upload path

text = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + " "

print("Characters:", len(text))
print(text[:800])


Characters: 133468
text
O my Luve's like a red, red rose
That’s newly sprung in June;
O my Luve's like the melodie
That’s sweetly play'd in tune.
As fair art thou, my bonnie lass,
So deep in luve am I:
And I will luve thee still, my dear,
Till a’ the seas gang dry:
Till a’ the seas gang dry, my dear,
And the rocks melt wi’ the sun:
I will luve thee still, my dear,
While the sands o’ life shall run.
And fare thee well, my only Luve
And fare thee well, a while!
And I will come again, my Luve,
Tho’ it were ten thousand mile.
The rose is red,
The violet's blue,
Sugar is sweet,
And so are you.
How do I love thee? Let me count the ways.
I love thee to the depth and breadth and height
My soul can reach, when feeling out of sight
For the ends of being and ideal grace.
I love thee to the level of every day's
Most qui


Text Cleaning

In [ ]:
text = text.lower()
text = re.sub(r'\n+', ' ', text)
text = re.sub(r'[^a-z\s]', '', text)
text = re.sub(r'\s+', ' ', text)

print(text[:500])


text o my luves like a red red rose thats newly sprung in june o my luves like the melodie thats sweetly playd in tune as fair art thou my bonnie lass so deep in luve am i and i will luve thee still my dear till a the seas gang dry till a the seas gang dry my dear and the rocks melt wi the sun i will luve thee still my dear while the sands o life shall run and fare thee well my only luve and fare thee well a while and i will come again my luve tho it were ten thousand mile the rose is red the vi


#Tokenization & Vocabulary

In [ ]:
words = text.split()

vocab = sorted(list(set(words)))
vocab_size = len(vocab)

word_to_ix = {w: i for i, w in enumerate(vocab)}
ix_to_word = {i: w for w, i in word_to_ix.items()}

print("Total words:", len(words))
print("Vocabulary size:", vocab_size)


Total words: 24677
Vocabulary size: 5440


###PART 1: RNN FROM SCRATCH (NUMPY)

In [ ]:
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Why = np.random.randn(output_size, hidden_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))
        outputs = []
        for x in inputs:
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            y = self.Why @ h + self.by
            outputs.append(y)
        return outputs

print("NumPy RNN implemented ✔")


NumPy RNN implemented ✔


PART 2: ONE-HOT ENCODING + PyTorch RNN

In [ ]:
SEQ_LEN = 5

X = []
Y = []

for i in range(len(words) - SEQ_LEN):
    X.append([word_to_ix[w] for w in words[i:i+SEQ_LEN]])
    Y.append(word_to_ix[words[i+SEQ_LEN]])

X = torch.tensor(X)
Y = torch.tensor(Y)

print(X.shape, Y.shape)


torch.Size([24672, 5]) torch.Size([24672])


One-Hot Encoding Function

In [ ]:
def one_hot_encode(x, vocab_size):
    return torch.eye(vocab_size)[x]


RNN Model (One-Hot Input)

In [ ]:
class RNN_OneHot(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(vocab_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

model_oh = RNN_OneHot(vocab_size, 128)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_oh.parameters(), lr=0.001)


Train One-Hot Model

In [ ]:
for epoch in range(5):
    optimizer.zero_grad()
    x_oh = one_hot_encode(X, vocab_size)
    outputs = model_oh(x_oh)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 8.6103
Epoch 2, Loss: 8.5853
Epoch 3, Loss: 8.5596
Epoch 4, Loss: 8.5309
Epoch 5, Loss: 8.4966


**PART 3: TRAINABLE WORD EMBEDDINGS**

In [ ]:
class RNN_Embedding(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

model_emb = RNN_Embedding(vocab_size, 50, 128)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_emb.parameters(), lr=0.001)


Train Embedding Model

In [ ]:
for epoch in range(5):
    optimizer.zero_grad()
    outputs = model_emb(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 8.6165
Epoch 2, Loss: 8.5827
Epoch 3, Loss: 8.5482
Epoch 4, Loss: 8.5118
Epoch 5, Loss: 8.4724


Generate Text

In [ ]:
def generate_text(model, start_words, n_words=20):
    model.eval()
    words_gen = start_words.copy()

    for _ in range(n_words):
        x = torch.tensor([[word_to_ix[w] for w in words_gen[-SEQ_LEN:]]])
        out = model(x)
        next_word = ix_to_word[out.argmax().item()]
        words_gen.append(next_word)

    return " ".join(words_gen)

print(generate_text(model_emb, words[:5]))


text o my luves like brute of the sin of the shore only the shore only the shore only the shore only the shore only
